<a href="https://colab.research.google.com/github/machine-perception-robotics-group/JDLALectureNotebooks/blob/master/notebooks/16_bi-directional_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 双方向LSTMによる品詞のタグ付け

---
## 目的
双方向LSTM (Bi-direcitional LSTM) を用いて英単語に対する品詞をタグ付け (POS tagging) を行う．


## 対応するチャプター
* 10.3: 双方向RNN

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from os.path import join
from time import time
import numpy as np
import json

import torch
import torch.nn as nn

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データのダウンロードとデータローダーの準備


### データのダウンロード
実習に必要なデータをダウンロードします．
下記のコードを実行してデータのダウンロードを行ってください．

In [ ]:
!wget http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/pos-tagging_data.zip
!unzip -q -o pos-tagging_data.zip
!ls ./pos-tagging_data

### データローダーの作成

データを読み込むためのデータローダーを定義します．


In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, root="./pos-tagging_data", train=True):
        super().__init__()

        if train:
            self.sentence = self.download_text_dataset(join(root, "train_sentence.txt"))
            self.label = self.download_text_dataset(join(root, "train_tags.txt"))
        else:
            self.sentence = self.download_text_dataset(join(root, "test_sentence.txt"))
            self.label = self.download_text_dataset(join(root, "test_tags.txt"))

        with open(join(root, "vocab.json")) as f:
            self.vocab = json.load(f)
            self.vocab = {v:k for k, v in self.vocab.items()}
        _n_vocab = len(self.vocab)
        self.vocab[_n_vocab] = "<PAD>"
        self.vocab_pad_id = _n_vocab
        self.n_vocab = len(self.vocab)
            
        with open(join(root, "tags.json")) as f:
            self.tags = json.load(f)
            self.tags = {v:k for k, v in self.tags.items()}
        _n_tags = len(self.tags)
        self.tags[_n_tags] = "<PAD>"
        self.tags_pad_id = _n_tags
        self.n_tags = len(self.tags)

        # 文章の長さの最大値を検索
        self.max_length = 0
        for s in self.sentence:
            if self.max_length < s.shape[0]:
                self.max_length = s.shape[0]

    def __getitem__(self, item):
        _s = self.sentence[item]
        _l = self.label[item]
        _s = np.pad(_s, (0, self.max_length - _s.shape[0]), constant_values=self.vocab_pad_id)
        _l = np.pad(_l, (0, self.max_length - _l.shape[0]), constant_values=self.tags_pad_id)
        return _s, _l.astype(np.int64)

    def __len__(self):
        return len(self.sentence)

    def download_text_dataset(self, input_filename):
        downloaded = []
        with open(input_filename) as f:
            for s in f.readlines():
                downloaded.append( np.array(list(map(int, s.strip().split(' '))), dtype=np.int32) )
        return downloaded

定義したデータローダーから，学習・評価用データセットを準備します．
また，呼び出したデータセットの情報を表示して，確認します．

In [ ]:
train_data = TextDataset(root="./pos-tagging_data", train=True)
test_data = TextDataset(root="./pos-tagging_data", train=False)

print("Train data -----------")
print("    Max length of sentence:", train_data.max_length)
print("    The number of vocabularies:", train_data.n_vocab)
print("    The number of tags:", train_data.n_tags)
print("    Padding ID for input sentence:", train_data.vocab_pad_id)
print("    Padding ID for output tag:", train_data.tags_pad_id)

print("Test data ------------")
print("    Max length of sentence:", test_data.max_length)
print("    The number of vocabularies:", test_data.n_vocab)
print("    The number of tags:", test_data.n_tags)
print("    Padding ID for input sentence:", test_data.vocab_pad_id)
print("    Padding ID for output tag:", test_data.tags_pad_id)

## ネットワークモデルの定義
Bidirectional LSTMを用いて，品詞タグ付けを行うためのネットワークを定義します．

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, n_vocab, n_tags, n_layers, n_units, padding_id):
        super().__init__()
        self.embed = nn.Embedding(n_vocab, n_units, padding_idx=padding_id)
        self.bi_lstm = nn.LSTM(n_units, n_units, num_layers=n_layers, batch_first=True, bidirectional=True)
        self.output = nn.Linear(2 * n_units, n_tags)

    def forward(self, xs):
        h = self.embed(xs)
        h, _ = self.bi_lstm(h)
        h = self.output(h)
        return h

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を0.01として引数に与えます．

In [ ]:
num_vocab = train_data.n_vocab
num_tags = train_data.n_tags

model = BiLSTM(num_vocab, num_tags, 2, 512, padding_id=train_data.vocab_pad_id)
if use_cuda:
    model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 学習

学習を実行します．

※ 学習には時間を要します．
演習時間の都合で十分な学習が行えない場合は，下記にある学習済みモデルを用いたテストで結果を確認してください．

In [ ]:
# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 64
epoch_num = 100

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion = criterion.cuda()

model.train()


start = time()
for epoch in range(1, epoch_num + 1):
    sum_loss = 0.0

    for input, label in train_loader:
        
        if use_cuda:
            input = input.cuda()
            label = label.cuda()

        y = model(input)

        loss = criterion(y.permute(0, 2, 1), label)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()

    print("epoch: {}, mean loss: {}, elapsed time: {}".format(epoch,
                                                              sum_loss/len(train_loader),
                                                              time() - start))

## テスト

学習後のネットワークを用いて，品詞のタグづけを行います．

ここではテストデータのうち10個の文章に対するタグづけの結果を示しています．

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

model.eval()

with torch.no_grad():
    for count, (input, label) in enumerate(test_loader):
        if use_cuda:
            input = input.cuda()
            label = label.cuda()

        y = model(input)

        pred = torch.argmax(y, dim=2)

        if use_cuda:
            input = input.data.cpu().numpy()
            pred = pred.data.cpu().numpy()
            label = label.data.cpu().numpy()

        input_sentence = [test_data.vocab[i] for i in input.flatten()]
        pred_tags = [test_data.tags[i] for i in pred.flatten()]
        true_tags = [test_data.tags[i] for i in label.flatten()]

        last_word_index = np.min(np.where(input.flatten() == test_data.vocab_pad_id))

        print("input sentence:", " ".join(input_sentence[:last_word_index]))
        print("predicted POS :", " ".join(pred_tags[:last_word_index]))
        print("true POS      :", " ".join(true_tags[:last_word_index]) + "\n")

        if count == 9:
            break

## テスト（学習済みモデル）

学習には時間を要するため，下記のコードでは学習済みのモデルを読み込んで，学習後のネットワークでの翻訳結果を確認します．
保存したモデルパラメータを読み込んで，テストデータの翻訳を行います．

In [ ]:
num_vocab = train_data.n_vocab
num_tags = train_data.n_tags
model_pretrain = BiLSTM(num_vocab, num_tags, 2, 512, padding_id=train_data.vocab_pad_id)
model_pretrain.load_state_dict(torch.load("pos-tagging_data/bilstm.pth"))
if use_cuda:
    model_pretrain.cuda()

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

model.eval()

with torch.no_grad():
    for count, (input, label) in enumerate(test_loader):
        if use_cuda:
            input = input.cuda()
            label = label.cuda()

        y = model_pretrain(input)

        pred = torch.argmax(y, dim=2)

        if use_cuda:
            input = input.data.cpu().numpy()
            pred = pred.data.cpu().numpy()
            label = label.data.cpu().numpy()

        input_sentence = [test_data.vocab[i] for i in input.flatten()]
        pred_tags = [test_data.tags[i] for i in pred.flatten()]
        true_tags = [test_data.tags[i] for i in label.flatten()]

        last_word_index = np.min(np.where(input.flatten() == test_data.vocab_pad_id))

        print("input sentence:", " ".join(input_sentence[:last_word_index]))
        print("predicted POS :", " ".join(pred_tags[:last_word_index]))
        print("true POS      :", " ".join(true_tags[:last_word_index]) + "\n")

        if count == 9:
            break